## Setting Up

In [1]:
import os

MODEL = os.getenv("OPENAI_MODEL")

In [2]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=MODEL
)

## Agent Runner

Running the Agent Asynchronously

`Runner.run()` runs the agent in async mode

In [3]:
result = await Runner.run(
    agent,
    "Tell me a short story"
)

result.final_output

'Once upon a time, in a small village nestled between rolling hills, there lived a young girl named Lila. Lila was known for her insatiable curiosity and love for exploring the nearby woods. Each day, she would venture deeper into the forest, collecting beautiful stones and observing the vibrant wildlife.\n\nOne sunny afternoon, as she wandered off the beaten path, Lila stumbled upon a hidden glade. In the center of the glade stood an ancient oak tree, its trunk wide and gnarled, with leaves that shimmered like emeralds in the sunlight. To her astonishment, she noticed a small, iridescent door at the base of the tree.\n\nWith her heart racing, Lila approached the door and, on a whim, knocked lightly. To her surprise, the door creaked open, revealing a tiny, shimmering world inside. A small voice greeted her, “Welcome, traveler! I’m Elyan, the guardian of this enchanted grove.”\n\nElyan explained that the glade was a sanctuary for all the magical creatures of the forest. However, it nee

### Async Streaming Mode

Start by creating a response using `Runner.run_streamed(...)`  \
Then iterate over it asynchronously

In [4]:
response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_67f0012024948191a6edc3040f1fb4cc0f03f95e296eb698', created_at=1743782176.0, error=None, incomplete_details=None, instructions="You're a helpful assistant", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=False, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reaso

## Events

The streaming response returns **Events**. This helps us filter the output to extract relevant information.

There are several events:
* RawEvent: Token output
* AgentUpdated: Agent creation
* StreamEvent: A message being completed
* 

In [5]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
     if event.type == "raw_response_event" and \
         isinstance(event.data, ResponseTextDeltaEvent):
             print(event.data.delta, end="", flush=True)

Once upon a time in a small village nestled between lush green hills, there lived a young girl named Lila. Lila had a curious mind and an adventurous spirit, always exploring the meadows and woods surrounding her home.

One sunny afternoon, while wandering deeper into the forest than usual, she stumbled upon an old, gnarled tree with a hollow trunk. Peering inside, she discovered a shimmering key resting on a bed of moss. Intrigued, Lila took the key and decided to find what it might unlock.

After searching for days, she noticed a hidden door in the base of a nearby hill, covered in vines and seemingly forgotten. With a heart racing with excitement, she inserted the key into the lock. To her amazement, the door creaked open, revealing a magical world full of vibrant creatures and fantastical landscapes.

In this enchanted realm, Lila befriended talking animals, danced with fairies, and even learned the art of cloud-painting from a wise old owl. Each adventure sparked her imagination a

## Tools

In [6]:
from agents import function_tool

@function_tool
def multiply(x: int, y: int) -> int:
    """Multiply 2 numbers to provide the product"""
    return x*y

In [7]:
agent = Agent(
    name="Assistant",
    instructions="""
        You're a helpful assistant who uses,
        provided tools to help whenever possible.,
        Do not rely on your own knowledge too much,
        instead use the tools to answer queries.
    """,
    model=MODEL,
    tools=[multiply]
)

In [8]:
response = Runner.run_streamed(
    agent,
    "What is the product of 4 and 100 ?"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="\n        You're a helpful assistant who uses,\n        provided tools to help whenever possible.,\n        Do not rely on your own knowledge too much,\n        instead use the tools to answer queries.\n    ", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='multiply', description='Multiply 2 numbers to provide the product', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}, 'required': ['x', 'y'], 'title': 'multiply_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x71e8678edbd0>, strict_json_schema=True)], mcp_servers=[], inpu

### Parsing Tool calling Events

In [9]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseTextDeltaEvent
)

response = Runner.run_streamed(
    starting_agent=agent,
    input="Multiply 4 with 7"
)

async for event in response.stream_events():
    
    if event.type == 'raw_response_event':
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # final answer
            print(event.data.delta, end="", flush=True)
    elif event.type == 'agent_updated_stream_event':
        # current agent
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == 'run_item_stream_event':
        if event.name == 'tool_called':
            # this is complete collection of tool call after the tokens indicating
            # tool call are completely streamed out
            print()
            print(f"> Tool called: {event.item.raw_item.name}")
            print(f"> Tool args: {event.item.raw_item.arguments}")
        elif event.name == 'tool_output':
            print(f"> Tool Output: {event.item.raw_item['output']}")
            
            

> Current Agent: Assistant
{"x":4,"y":7}
> Tool called: multiply
> Tool args: {"x":4,"y":7}
> Tool Output: 28
The product of 4 and 7 is 28.

## Guardrails

In [10]:
from pydantic import BaseModel

# define structure for output guardrail
class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str
    
# an agent that checks if user is asking about political opinions
politics_agent = Agent(
    name="Politics check",
    instructions="Check if the user is asking you about political opinions.",
    output_type=GuardrailOutput
)

In [11]:
query = "What do you think about BJP in India ?"

result = await Runner.run(starting_agent=politics_agent, input=query)
result

RunResult(input='What do you think about BJP in India ?', new_items=[MessageOutputItem(agent=Agent(name='Politics check', instructions='Check if the user is asking you about political opinions.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=<class '__main__.GuardrailOutput'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_67f0013c7b6c8191b154d6ff08acfb03013c60bd13899484', content=[ResponseOutputText(annotations=[], text='{"is_triggered":true,"reasoning":"The user is asking for an opinion on a political party, BJP, in India, which falls under the category of political opinions."}', type='output_text')], role='assistant', status='completed', type='message')

In [12]:
result.final_output

GuardrailOutput(is_triggered=True, reasoning='The user is asking for an opinion on a political party, BJP, in India, which falls under the category of political opinions.')

### Integrating the guardrail with our agent

For the Input Guardrail to work on the agent, we need to pass 
the logic to a single function

* Input parameters include: `ctx` (Context), `agent`, `input`
* Output must be a `GuardrailFunctionOutput` object

In [13]:
from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail
)

@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str
) -> GuardrailFunctionOutput:
    
    # run agent to check if guardrail is triggered
    response = await Runner.run(starting_agent=politics_agent, input=input)
    # format response into GuardrailFunctionOutput
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered
    ) 


In [14]:
# Redefine agent with input guardrail
agent = Agent(
    name="Assistant",
    instructions="""
        You're a helpful assistant who uses,
        provided tools to help whenever possible.,
        Do not rely on your own knowledge too much,
        instead use the tools to answer queries.
    """,
    model=MODEL,
    tools=[multiply],
    input_guardrails=[politics_guardrail]
)

In [17]:
result = await Runner.run(
    starting_agent=agent,
    input="multiply 4 with 7"
)

result.final_output

'The product of 4 and 7 is 28.'

In [18]:
result = await Runner.run(
    starting_agent=agent,
    input="What do you think about the BJP in India"
)

result.final_output

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

The Guardrail got triggered ! A `tripwire` is like a fail-safe for Guardrails. 

In some cases,
when the guardrail is triggered, the system needs to **hard** stop. This is where tripwires can help 

## Conversational Agent

We can use `.to_input_list()` on our return object to create a list of messages from the output, that can be used further.

In [19]:
result = await Runner.run(
    starting_agent=agent,
    input="Remember the number 42 please"
)

result.final_output

"I can't store information for future use, but I can remember details during our current chat! If you need to refer to the number 42 later in this conversation, feel free to ask!"

In [20]:
result.to_input_list()

[{'content': 'Remember the number 42 please', 'role': 'user'},
 {'id': 'msg_67f0146aa6308191ae59d964ef0b6eca06c69f5e3b0906f3',
  'content': [{'annotations': [],
    'text': "I can't store information for future use, but I can remember details during our current chat! If you need to refer to the number 42 later in this conversation, feel free to ask!",
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [21]:
result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "Multiply the last number with 6"}
    ]
)

result.final_output

'The result of multiplying 42 by 6 is 252.'